<a href="https://colab.research.google.com/github/Faith-Cherotich/Multimodal_LLM/blob/main/Multimodal_LLM(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install langchain-groq langchain sentence-transformers langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
%pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 94.8 MB/s eta 0:00:00


In [3]:
# Importing libraries
import fitz # PyMuPDF
from langchain_core.documents import Document
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import numpy as np
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage
import os
import base64
import io
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

In [4]:
# Loading the clip models
# Processor & model
import os
from dotenv import load_dotenv
load_dotenv("/content/OPENAI_API_KEY.env")

# Setting up the environment
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [5]:
# Initializing the Clip Model for unified embeddings
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [6]:
# Embedding images- converting to image vector
def embed_image(image_data):
  """Embed image using CLIP"""
  if isinstance(image_data, str):
    image = Image.open(image_data).convert("RGB")
  else: # If PIL Image
    image = image_data

  inputs=clip_processor(images=image,return_tensors="pt")
  with torch.no_grad():
    features = clip_model.get_image_features(**inputs)
    # Normalizing embeddings to unit vector
    features=features/features.norm(dim=-1, keepdim=True)
    return features.squeeze().numpy()

In [7]:
# Embedding text- getting text features
def embed_text(text):
  """Embed text using CLIP"""
  inputs = clip_processor(
      text=text,
      return_tensors="pt",
      padding=True,
      truncation=True,
      max_length=77 # max token length
  )
  with torch.no_grad():
    features=clip_model.get_text_features(**inputs)
    # Normalizing embeddings
    features=features/features.norm(dim=-1, keepdim=True)
    return features.squeeze().numpy()

In [8]:
# Read pdf document and extract images
pdf_path="/content/Kenya_GDP_Q1_2024.pdf"
doc=fitz.open(pdf_path)

In [9]:
# Storing documents and embeddings
all_docs=[]
all_embeddings=[]
image_data_store={} # actual image data for llm

In [10]:
# Text Splitter
splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [11]:
doc

Document('/content/Kenya_GDP_Q1_2024.pdf')

In [12]:
# Text-Text vectors
for i, page in enumerate(doc):
  # Processing text
  text=page.get_text()
  if text.strip():
    # Temporary document for splitting
    temp_doc = Document(page_content=text, metadata={"page":i, "type": "text"})
    text_chunks = splitter.split_documents([temp_doc])

    # Embedding each chunk using CLIP
    for chunk in text_chunks:
      embedding = embed_text(chunk.page_content)
      all_embeddings.append(embedding)
      all_docs.append(chunk)

  # Processing images
  # Convert PDF image to PIL format, store as base64 for GPT-4V which needs base64 images, create clip embedding for retrieval
  for img_index, img in enumerate(page.get_images(full=True)):
    try:
      xref = img[0]
      base_image = doc.extract_image(xref)
      image_bytes = base_image["image"]

      # Converting to PIL image
      pil_image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

      # Unique Identifier
      image_id = f"page_{i}_img_{img_index}"

      # Store image as base64 for use with GPT-4v
      buffered = io.BytesIO()
      pil_image.save(buffered, format="PNG")
      img_base64 = base64.b64encode(buffered.getvalues()).decode()
      image_data_store[image_id] = img_base64

      # Embed image using clip
      embedding = embed_image(pil_image)
      all_embeddings.append(embedding)

      # Create document for image
      image_doc = Document(
          page_content=f"[Image: {image_id}]",
          metadata={"page": i, "type": "image", "image_id": image_id}
      )
      all_docs.append(image_doc)
    except Exception as e:
      print(f"Error processing image {img_index} on page {i}:{e}")
      continue

doc.close()

Error processing image 0 on page 0:'_io.BytesIO' object has no attribute 'getvalues'


In [13]:
# processing images
# Convert PDF image to PIL format, store as base64 for GPT-4V which needs base64 images, create clip embedding for retrieval


In [14]:
all_embeddings

[array([ 2.14804262e-02, -4.83991913e-02, -1.19440248e-02, -7.35645043e-03,
        -1.64050201e-03,  2.36823391e-02,  1.18401153e-02,  1.25025898e-01,
         3.28750722e-02,  3.33376080e-02, -8.85537080e-03, -2.69455612e-02,
        -5.97332753e-02,  1.13359913e-02,  6.17285334e-02,  4.11608577e-04,
         4.81003188e-02,  7.11986155e-04, -1.33029057e-03,  7.47829378e-02,
        -7.22074765e-04,  2.16315370e-02,  2.27272436e-02,  1.73413642e-02,
        -6.30982080e-03,  1.17607675e-02,  5.46328630e-03, -6.33041468e-03,
         2.39555854e-02, -3.03798877e-02, -2.91078687e-02, -5.87149858e-02,
        -2.72939187e-02, -5.64253144e-02,  1.02648102e-02,  5.06446436e-02,
         2.50197891e-02, -3.45076285e-02,  5.80246449e-02,  2.16357387e-03,
        -5.23959361e-02,  1.69057027e-02, -1.25310372e-03, -4.05205152e-04,
        -4.32956629e-02,  1.49820307e-02,  6.14709109e-02, -2.20390186e-02,
        -2.95716580e-02,  9.54207499e-03,  2.53251623e-02, -1.01490170e-02,
         2.5

In [15]:
embeddings_array = np.array(all_embeddings)
embeddings_array

array([[ 0.02148043, -0.04839919, -0.01194402, ..., -0.01826797,
        -0.02148849,  0.01810989],
       [-0.01288443,  0.00114727, -0.00503997, ..., -0.04131815,
        -0.01087308,  0.0202192 ],
       [ 0.01573821,  0.00481264, -0.00812808, ..., -0.05149907,
        -0.00455549, -0.01757021],
       ...,
       [ 0.02676501,  0.01969731, -0.01057212, ..., -0.00028869,
        -0.02610743, -0.01408502],
       [ 0.02415547, -0.00191386, -0.00260592, ..., -0.01641496,
        -0.01473243, -0.00729489],
       [ 0.02269348, -0.00137373, -0.00898131, ..., -0.03102647,
        -0.01630017, -0.00845137]], dtype=float32)

In [16]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 17.9 MB/s eta 0:00:00


In [17]:
# Creating a unified FAISS vector store with CLIP embeddings
import faiss
embeddings_array = np.array(all_embeddings)

# custom FAISS index since we have precomputed embeddings
vector_store = FAISS.from_embeddings(
    text_embeddings=[(doc.page_content, emb) for doc, emb in zip(all_docs, embeddings_array)],
    embedding=None, # Precomputed embeddings
    metadatas=[doc.metadata for doc in all_docs]
)
vector_store

In [18]:
%pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 3.4 MB/s eta 0:00:00


In [19]:
# Initialize the llm
llm = init_chat_model("openai:gpt-4o")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7c1eff69bf80>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7c1eff6b7d40>, root_client=<openai.OpenAI object at 0x7c1fa6928140>, root_async_client=<openai.AsyncOpenAI object at 0x7c1eff7cd9a0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [20]:
# Initializing the llm
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7c1eff6b7710>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7c1eff5c3ad0>, root_client=<openai.OpenAI object at 0x7c1eff693740>, root_async_client=<openai.AsyncOpenAI object at 0x7c1eff6b60f0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [21]:
# Rag Pipeline
def retrieve_multimodal(query, k=5):
  """unified retrieval using CLIP embeddings for both text and images."""
  # Embed query using clip
  query_embedding = embed_text(query)

  # Search unified vector store based on query embeddings
  results = vector_store.similarity_search_by_vector(
      embedding=query_embedding,
      k=k
  )

  return results

In [22]:
def create_multimodal_message(query, retrieved_docs):
  """Create a message with both text and images for GPT-4V"""
  content=[]

  # Add the query
  content.append({
      "type": "text",
      "text": f"Question: {query}\n\nContext:\n"
  })

  # Separate text and image documents
  text_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "text"]
  image_docs = [doc for doc in retrieved_docs if doc.metadata.get("type") == "image"]

  # Adding the text context
  if text_docs:
    text_context = "\n\n".join([
        f"[Page {doc.metadata['page']}]: {doc.page_content}"
        for doc in text_docs
    ])
    content.append({
        "type": "text",
        "text": f"Text excerpts:\n{text_context}\n"
    })

  # Adding images
  for doc in image_docs:
    image_id = doc.metadata.get("image_id")
    if image_id and image_id in image_data_store:
      content.append({
        "type": "text",
        "text": f"\n[Image from page {doc.metadata['page']}]:\n"
      })
      content.append({
        "type": "image_url",
        "image_url": {
            "url": f"data:image/png:base64,{image_data_store[image_id]}"
        }
      })
  # Adding instruction
  content.append({
      "type": "text",
      "text": "\n\nPlease answer the question based on the provided text and images."
  })

  return HumanMessage(content=content)

In [23]:
# Intergrating into a rag pipeline
def multimodal_pdf_rag_pipeline(query):
  """Main pipeline for multimodal RAG."""
  # Retrieving relevant documents
  context_docs = retrieve_multimodal(query, k=5)

  # Creating a multimodal message
  message = create_multimodal_message(query, context_docs)

  # Response from GPT-4VV
  response = llm.invoke([message])

  # Printing retrieved context info (entire info)
  print(f"\nRetrieved {len(context_docs)} documents:")
  for doc in context_docs:
    doc_type = doc.metadata.get("type", "unknown")
    page=doc.metadata.get("page", "?")
    if doc_type == "text":
      preview = doc.page_content[:100] + "..." if len(doc.page_content)>100 else doc.page_content
      print(f" - Text from page {page}: {preview}")
    else:
      print(f" - Image from page {page}")
  print("\n")

  return response.content